# Vis 5
## Data Processing

In [1]:
import altair as alt
from altair import datum
from vega_datasets import data
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim

mapDataWithCountryNames= alt.topo_feature("https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json",'name')
mapData = alt.topo_feature("https://vega.github.io/vega/data/world-110m.json",'countries')

locs = pd.read_csv('./pleiades-locations-latest.csv')
places = pd.read_csv('./pleiades-places-latest.csv')
names = pd.read_csv('./pleiades-names-latest.csv')

capitals = pd.read_json('http://techslides.com/demos/country-capitals.json')



print("locs.shape = ",locs.shape)
print("places.shape = ",places.shape)
print("names.shape = ",names.shape)
print("capitals.shape = ",capitals.shape)

locs.shape =  (40463, 25)
places.shape =  (37737, 26)
names.shape =  (34133, 26)
capitals.shape =  (245, 6)


## Data Prep

In [2]:
settlementFilter=locs.featureType==("settlement")
settlementData = locs[settlementFilter]

bathFilter=locs.featureType==("bath")
bathData = locs[bathFilter]

empireDataS = ((settlementData.timePeriods =="R")|(settlementData.timePeriods =="HR")|(settlementData.timePeriods =="HRL")
            |(settlementData.timePeriods =="RL")|(settlementData.timePeriods =="R")) 
empireSettlementPoints = settlementData[empireDataS]

empireDataB = ((bathData.timePeriods =="R")|(bathData.timePeriods =="HR")|(bathData.timePeriods =="HRL")
            |(bathData.timePeriods =="RL")|(bathData.timePeriods =="R")) 
empireBathPoints = bathData[empireDataB]


In [3]:

alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

## Visulisation

In [4]:
#prerequisites
centerVal = [30,45]
scaleVal = 500


# world map
background = alt.Chart(mapData).mark_geoshape(
    fill = '#fffff',
    stroke = '#AAA9A9').project(
    center = centerVal,
    scale=scaleVal,
).properties(
    width=800,
    height=500
)

#slider
slider  = alt.binding_range(min= 300, max=700, step=100)
selectorMaxDate  = alt.selection_single(name="set", fields=['maxDate'],
                                   bind=slider, init={'maxDate': 300})

# points on background
romePSettlement = alt.Chart(empireSettlementPoints).mark_point(
    color='#d1081f',
    size=12
).project(
    center = centerVal,
    scale=scaleVal,
).encode(
    longitude='reprLong',
    latitude='reprLat',
    tooltip=['reprLatLong','maxDate','minDate','timePeriods','description'],
    size=alt.Size('minDate:N',scale=alt.Scale(range=(6,30)))
).add_selection(
    selectorMaxDate 
).transform_filter(
    datum.maxDate < selectorMaxDate.maxDate
)


# capital text
capitalsText = alt.Chart(capitals).mark_text(
    size=6.5, 
    font='calibri'
).project(
    center = centerVal,
    scale=scaleVal,
).encode(
    longitude='CapitalLongitude:Q',
    latitude='CapitalLatitude:Q',
    text = 'CapitalName'
)

mainMap = (background + romePSettlement  + capitalsText) 
mainMap



alt.LayerChart(...)

In [5]:
#prerequisites
centerVal = [30,45]
scaleVal = 500


# world map
background = alt.Chart(mapData).mark_geoshape(
    fill = '#fffff',
    stroke = '#AAA9A9').project(
    center = centerVal,
    scale=scaleVal,
).properties(
    width=800,
    height=500
)

#slider
slider  = alt.binding_range(min= 300, max=700, step=100)
selectorMaxDate  = alt.selection_single(name="set", fields=['maxDate'],
                                   bind=slider, init={'maxDate': 300})

# points on background
romePSettlement = alt.Chart(empireSettlementPoints).mark_point(
    color='#d1081f',
    size=12
).project(
    center = centerVal,
    scale=scaleVal,
).encode(
    longitude='reprLong',
    latitude='reprLat',
    tooltip=['reprLatLong','maxDate','minDate','timePeriods','description'],
    size=alt.Size('minDate:N',scale=alt.Scale(range=(6,30)))
).add_selection(
    selectorMaxDate 
).transform_filter(
    datum.maxDate < selectorMaxDate.maxDate
)


romePBath= alt.Chart(empireBathPoints).mark_point(
    color='#00ff59',
    size=12
).project(
    center = centerVal,
    scale=scaleVal,
).encode(
    longitude='reprLong',
    latitude='reprLat',
    tooltip=['reprLatLong','maxDate','minDate','timePeriods','description'],
    size=alt.Size('minDate:N',scale=alt.Scale(range=(6,30)))
).add_selection(
    selectorMaxDate 
).transform_filter(
    datum.maxDate < selectorMaxDate.maxDate
)
# capital text
capitalsText = alt.Chart(capitals).mark_text(
    size=6.5, 
    font='calibri'
).project(
    center = centerVal,
    scale=scaleVal,
).encode(
    longitude='CapitalLongitude:Q',
    latitude='CapitalLatitude:Q',
    text = 'CapitalName'
)

mainMap = (background  + romePSettlement+romePBath + capitalsText) 
mainMap



alt.LayerChart(...)